# Description

**TODO**: should probably be moved to preprocessing folder.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf
from multiplier import MultiplierProjection

# Settings

In [3]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [4]:
OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
display(OUTPUT_DATA_DIR)
OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data')

In [5]:
OUTPUT_RAW_DATA_DIR = Path(OUTPUT_DATA_DIR, "raw")
display(OUTPUT_RAW_DATA_DIR)
OUTPUT_RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw')

In [6]:
OUTPUT_PROJ_DATA_DIR = Path(OUTPUT_DATA_DIR, "proj")
display(OUTPUT_PROJ_DATA_DIR)
OUTPUT_PROJ_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj')

# Load PhenomeXcan results

In [7]:
input_file_list = [
    conf.PHENOMEXCAN["SMULTIXCAN_MASHR_ZSCORES_FILE"],
    Path(
        conf.PHENOMEXCAN["SPREDIXCAN_MASHR_ZSCORES_FOLDER"],
        "most_signif",
        "spredixcan-most_signif.pkl",
    ),
]

In [8]:
# add S-PrediXcan results for each tissue
input_file_list = input_file_list + [
    f
    for f in Path(conf.PHENOMEXCAN["SPREDIXCAN_MASHR_ZSCORES_FOLDER"], "pkl").glob(
        "*.pkl"
    )
]

In [9]:
_tmp = len(input_file_list)
display(_tmp)
assert _tmp == 51

51

In [10]:
for input_file in input_file_list:
    print(input_file.name)

    # read data
    phenomexcan_data = pd.read_pickle(input_file)
    print(f"  shape: {phenomexcan_data.shape}")

    assert phenomexcan_data.index.is_unique
    assert phenomexcan_data.columns.is_unique

    phenomexcan_data = phenomexcan_data.dropna(how="any")
    print(f"  shape (no NaN): {phenomexcan_data.shape}")
    assert not phenomexcan_data.isna().any().any()

    output_file = Path(OUTPUT_RAW_DATA_DIR, f"{input_file.stem}-data.pkl").resolve()
    print(f"  saving to: {str(output_file)}")
    phenomexcan_data.to_pickle(output_file)

    # project
    print("  projecting...")
    mproj = MultiplierProjection()
    phenomexcan_projection = mproj.transform(phenomexcan_data)
    print(f"    shape: {phenomexcan_projection.shape}")

    output_file = Path(
        OUTPUT_PROJ_DATA_DIR, f"{input_file.stem}-projection.pkl"
    ).resolve()
    print(f"    saving to: {str(output_file)}")
    phenomexcan_projection.to_pickle(output_file)

    print("")

smultixcan-mashr-zscores.pkl
  shape: (22515, 4091)
  shape (no NaN): (22152, 4091)
  saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw/smultixcan-mashr-zscores-data.pkl
  projecting...
    shape: (987, 4091)
    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj/smultixcan-mashr-zscores-projection.pkl

spredixcan-most_signif.pkl
  shape: (22515, 4091)
  shape (no NaN): (22152, 4091)
  saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw/spredixcan-most_signif-data.pkl
  projecting...
    shape: (987, 4091)
    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj/spredixcan-most_signif-projection.pkl

spredixcan-mashr-zscores-Adipose_Subcutaneous.pkl
  shape: (14720, 4091)
  shape (no NaN): (14063, 4091)
  saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw/spredixca